In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pathlib import Path


# ✅ Use Headless Mode (Runs in Background)
options = webdriver.ChromeOptions()
options.add_argument("--headless")  
options.add_argument("--disable-gpu")  
options.add_argument("--window-size=1920,1080")  
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# ✅ Open LinkedIn Job Search Page (Without Login)
driver = webdriver.Chrome(options=options)
driver.get("https://www.linkedin.com/jobs/search?keywords=Software%20Engineer")
time.sleep(5)  # Wait for page to load

job_data = []

# ✅ Scrape Job Listings
jobs = driver.find_elements(By.CLASS_NAME, "base-card")  # LinkedIn job elements

for job in jobs[:5]:  # Get only the first 5 jobs
    try:
        title = job.find_element(By.CLASS_NAME, "base-search-card__title").text
        company = job.find_element(By.CLASS_NAME, "base-search-card__subtitle").text
        location = job.find_element(By.CLASS_NAME, "job-search-card__location").text
        job_url = job.find_element(By.TAG_NAME, "a").get_attribute("href")  # Job link

        job_data.append({
            "Title": title,
            "Company": company,
            "Location": location,
            "Job URL": job_url
        })
    except:
        continue

# ✅ Save Jobs to CSV
df = pd.DataFrame(job_data)
ROOT_DIR = Path.cwd()
df.to_csv(ROOT_DIR / "linkedin_jobs.csv", index=False)

print("✅ Scraping Completed! Check 'linkedin_jobs.csv'")
driver.quit()


✅ Scraping Completed! Check 'linkedin_jobs.csv'


In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from pathlib import Path

# ✅ Setup Chrome Options (Headless Mode)
options = webdriver.ChromeOptions()
options.add_argument("--headless")  
options.add_argument("--disable-gpu")  
options.add_argument("--window-size=1920,1080")  
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# ✅ Start WebDriver
driver = webdriver.Chrome(options=options)

# ✅ Open LinkedIn Job Search Page (No Login Required)
url = "https://www.linkedin.com/jobs/search/?currentJobId=4130519571&keywords=Data%20Scientist"
driver.get(url)
time.sleep(5)  # Wait for page to load

job_data = []

# ✅ Get All Job Listings
jobs = driver.find_elements(By.CLASS_NAME, "base-card")  

if not jobs:
    print("❌ No jobs found! LinkedIn might have blocked the scraper.")
else:
    for job in jobs[:5]:  # Limit to 5 jobs
        try:
            # ✅ Click Job to Open Details
            driver.execute_script("arguments[0].click();", job)
            time.sleep(3)  # Allow time for job details to load

            # ✅ Extract Job Details
            title = job.find_element(By.CLASS_NAME, "base-search-card__title").text
            company = job.find_element(By.CLASS_NAME, "base-search-card__subtitle").text
            location = job.find_element(By.CLASS_NAME, "job-search-card__location").text
            job_url = job.find_element(By.TAG_NAME, "a").get_attribute("href")

            # ✅ Wait for Job Description to Load
            try:
                description = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "jobs-description-content__text"))
                ).text
            except:
                description = "Not Available"

            # ✅ Extract Additional Info (If Available)
            try:
                experience = driver.find_element(By.XPATH, "//span[contains(text(),'Experience')]//following-sibling::span").text
            except:
                experience = "Not Mentioned"

            try:
                job_type = driver.find_element(By.XPATH, "//span[contains(text(),'Employment type')]//following-sibling::span").text
            except:
                job_type = "Not Mentioned"

            try:
                salary = driver.find_element(By.XPATH, "//span[contains(text(),'Pay')]//following-sibling::span").text
            except:
                salary = "Not Mentioned"

            # ✅ Store Job Data
            job_data.append({
                "Title": title,
                "Company": company,
                "Location": location,
                "Experience": experience,
                "Job Type": job_type,
                "Salary": salary,
                "Description": description,
                "Job URL": job_url
            })

            print(f"✅ Scraped: {title} at {company}")

        except Exception as e:
            print(f"⚠️ Error scraping job: {e}")

# ✅ Save Data to CSV
df = pd.DataFrame(job_data)
ROOT_DIR = Path.cwd()
csv_path = ROOT_DIR / "linkedin_data.csv"
df.to_csv(csv_path, index=False)

print(f"✅ Scraping Completed! Check '{csv_path}'")
driver.quit()


✅ Scraped: Data Scientist (L6) - Product at Netflix
✅ Scraped: Data Scientist - Healthcare & Clinical Data (Remote) at Walgreens
✅ Scraped: Data Scientist, Growth at Notion
✅ Scraped: Data Scientist, Growth at Notion
✅ Scraped:  at 
✅ Scraping Completed! Check '/home/h2/linkedin_data.csv'
